In [1]:
import numpy as np
import pandas as pd
import glob
from time import sleep
from tqdm import tqdm
import time
import signal
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from time import sleep
import os

In [12]:
site="https://powszechswiat.pl/user"
query="kościół +wirus"

In [21]:
d = webdriver.Chrome()

# d.get(site)

# sleep(60)

# d.get('https://www.tygodnikpowszechny.pl/')

# d.find_element(By.XPATH, '//div[@id="edit-actions"]').click()

d.get(f"https://www.tygodnikpowszechny.pl/search/site/{query}")

pagesMax=d.find_element(By.XPATH, '//li[@class="pager-last last"]/a').get_attribute('href')
pagesMax=int(pagesMax.split('=')[-1])
print(f'Found {pagesMax} pages')

links=[]

els=d.find_elements(By.XPATH, '//li[@class="search-result"]//a')
for el in els:
    links.append(el.get_attribute('href'))

for page in tqdm(range(pagesMax)):
    site=f"https://www.tygodnikpowszechny.pl/search/site/{query}?page={page+1}"
    d.get(site)
    els=d.find_elements(By.XPATH, '//li[@class="search-result"]//a')
    for el in els:
        links.append(el.get_attribute('href'))

Found 3 pages


100%|██████████| 3/3 [00:05<00:00,  1.72s/it]


In [22]:
for link in links:
    mjio

['https://www.tygodnikpowszechny.pl/wirus-i-wielka-tajemnica-wiary-163236',
 'https://www.tygodnikpowszechny.pl/wirus-uderza-w-finanse-kosciola-162781',
 'https://www.tygodnikpowszechny.pl/wirus-za-spizowa-brama-162499',
 'https://www.tygodnikpowszechny.pl/kosciol-w-skrocie-163720',
 'https://www.tygodnikpowszechny.pl/bylam-przy-poczatku-i-koncu-163080',
 'https://www.tygodnikpowszechny.pl/koniec-zycia-w-iluzji-163963',
 'https://www.tygodnikpowszechny.pl/kot-eliota-166438',
 'https://www.tygodnikpowszechny.pl/moja-i-twoja-nadzieja-163133',
 'https://www.tygodnikpowszechny.pl/bog-uchroni-nas-przed-zaraza-163307',
 'https://www.tygodnikpowszechny.pl/juz-od-srody-22-kwietnia-podwojne-wydanie-tygodnika-powszechnego-na-majowke-a-w-nim-163038',
 'https://www.tygodnikpowszechny.pl/idzie-grozne-149169',
 'https://www.tygodnikpowszechny.pl/kosciol-postawiony-pod-sciana-16905',
 'https://www.tygodnikpowszechny.pl/pandemia-covid19-serwis-specjalny',
 'https://www.tygodnikpowszechny.pl/przedluzon

In [15]:
els

[]